In [26]:
%reload_ext autoreload
%autoreload 2
import json
from ext.LIWClib.EmotionsLIWClib import *
# from ext.TextProcessingLib.TextProcessing import *
# from data.YelpDataLoader import *
from ext.GensimSemanticLib.GensimStatisticalProcessing import *

In [27]:
myLIWC = LIWCObj()
myLIWC.build_model('/Users/trevi/Projects/libs/LIWClib/dictionary/LIWC2007_English100131.dic')
myLIWC.unittest()

PASSED - ({'posemo': 0, 'negemo': 3}) This pasta sucks, it is made by shit. I don't like this pizza.
PASSED - ({'posemo': 0, 'negemo': 0}) Questa passata fa schifo!
PASSED - ({'posemo': 1, 'negemo': 2}) This restaurant has all my love but the food is not good and the service is terrible!


In [130]:
M0 = 'data/food_dataset.w2v/food_bbc_1count_100size.model'
M4 = 'data/food_dataset.w2v/w2v-food-bbc_menupages-1count_300size.notstem.model'
M5 = 'data/food_dataset.w2v/w2v-food-bbc_menupages-3count_300size.notstem.model'
gs0 = GensimCore(); gs0.load_model( M0 )
gs4 = GensimCore(); gs4.load_model( M4 )
gs5 = GensimCore(); gs5.load_model( M5 )

In [109]:
gs0.model.most_similar(positive=['wine'], negative=[])
gs0.model.most_similar(positive=['carbonara','pesto'], negative=['meat'])

[('lasagn', 0.8531348705291748),
 ('challeng', 0.7936052083969116),
 ('rick', 0.7791990041732788),
 ('glass', 0.7713647484779358),
 ('flottant', 0.7638132572174072),
 ('feel', 0.7623695135116577),
 ('langoustin_claw', 0.7620701789855957),
 ('effort', 0.7575380802154541),
 ('dian', 0.7571722269058228),
 ('stein', 0.7553558349609375)]

In [142]:
ss = "This pasta quattro formaggi sucks, it is made by shit. I don't like this pizza marinara."
ss_proc = preprocess_pipeline(ss, stemmer_type=None, do_remove_stopwords=True, noAccents=True, return_as_str=True)
print ss_proc
#
found_food = False
l_words = tokenize2words(ss_proc)
for i in range(1,len(l_words)):
    ## get food items
    pw = l_words[i-1]
    cw = l_words[i]
    try:
        print 'sim(%s,%s):' %(pw,cw), gs4.model.similarity(pw, cw)
        print '"%s"' %cw
        print gs4.model.most_similar(positive=[cw])[:3]
    except:
#         print '- not found'
        continue
if found_food:
    ## get sentimens
    print myLIWC.getCategoriesFromText(ss_proc)

pasta quattro formaggi sucks made shit dont like pizza marinara
sim(pasta,quattro): 0.523518417298
"quattro"
[('san', 0.744605541229248), ('ankake', 0.6891857981681824), ('santa', 0.6820037961006165)]
sim(quattro,formaggi): 0.59263812548
"formaggi"
[('daniele', 0.9198753237724304), ('prosciutto_month', 0.7697910666465759), ('tamales_oaxaquenos', 0.7602232098579407)]
sim(formaggi,sucks): sim(sucks,made): sim(made,shit): sim(shit,dont): sim(dont,like): 0.768308795026
"like"
[('boozesoaked', 0.9256796836853027), ('complemented', 0.9230676293373108), ('vilest', 0.9187042713165283)]
sim(like,pizza): -0.14485438334
"pizza"
[('pizza_burger', 0.8487417697906494), ('pepperoni', 0.7726970911026001), ('rustica', 0.7643126249313354)]
sim(pizza,marinara): 0.47269333301
"marinara"
[('bufala', 0.826797604560852), ('penne', 0.809898853302002), ('mozzarella_sticks', 0.8022845983505249)]


In [184]:
from nltk import pos_tag, ne_chunk
for sent in tokenize2sentences2words(ss):
    l_words = remove_stopwords(sent)
    ## select only nouns from l_words
    print pos_tag( l_words )
    l_nouns = [word for word,pos in pos_tag( l_words ) if pos == 'NN']
    ## match each word with food
    matches = ''
    topjac = -1
    for fw in food.keys():
      jac = jaccard_distance( l_nouns, fw.split(' ') )
      if jac > topjac:
        topjac = jac        
        matches = fw
    ## check matches
    print l_words, '-> (%.2f)' %topjac, matches
    print myLIWC.getCategoriesFromText(' '.join(sent))

[('pasta', 'NN'), ('quattro', 'NN'), ('formaggi', 'NN'), ('sucks', 'NNS'), ('made', 'VBN'), ('shit', 'NN')]
['pasta', 'quattro', 'formaggi', 'sucks', 'made', 'shit'] -> (0.50) quattro formaggi
{'posemo': 0, 'negemo': 2}
[('dont', 'NN'), ('like', 'IN'), ('pizza', 'NN'), ('marinara', 'NN')]
['dont', 'like', 'pizza', 'marinara'] -> (0.33) pizza
{'posemo': 0, 'negemo': 1}


In [33]:
## Playing with TextBlob
# http://textblob.readthedocs.org/en/dev/
from textblob import TextBlob
text = "This pasta quattro formaggi sucks, it is made by shit. I don't like this pizza marinara."
blob = TextBlob(text)
print blob.detect_language()
print blob.tags           # [('The', 'DT'), ('titular', 'JJ'),
                    #  ('threat', 'NN'), ('of', 'IN'), ...]

print blob.noun_phrases   # WordList(['titular threat', 'blob',
                    #            'ultimate movie monster',
                    #            'amoeba-like mass', ...])

for sentence in blob.sentences:
    print sentence
    print '->', sentence.sentiment.polarity
    print '->', myLIWC.getCategoriesFromText( str(sentence) )
# 0.060
# -0.341

blob.translate(to="es")  # 'La amenaza titular de The Blob...'

 en
[(u'This', u'DT'), (u'pasta', u'NN'), (u'quattro', u'NN'), (u'formaggi', u'NN'), (u'sucks', u'VBZ'), (u'it', u'PRP'), (u'is', u'VBZ'), (u'made', u'VBN'), (u'by', u'IN'), (u'shit', u'NN'), (u'I', u'PRP'), (u'do', u'VBP'), (u'n', u'NN'), (u"'", u'POS'), (u't', u'NN'), (u'like', u'IN'), (u'this', u'DT'), (u'pizza', u'NN'), (u'marinara', u'NN')]
[u'pasta quattro formaggi sucks', u'pizza marinara']
This pasta quattro formaggi sucks, it is made by shit.
-> -0.25
-> {'posemo': 0, 'negemo': 2}
I don't like this pizza marinara.
-> 0.0
-> {'posemo': 0, 'negemo': 1}


TextBlob("Esta formaggi quattro pastas chupa , se hace por una mierda. No me gusta este marinara pizza.")

In [236]:
'''
Load Business and Review processed data
'''
YBFILE = 'data/yelp_restaurants_business.p'
YRFILE = 'data/yelp_restaurants_review.p'
print 'Loading Business Container (%s)' %YBFILE
businessContainer = pickle.load( open( YBFILE, "rb" ) )
print '\t-> loaded %d business' %businessContainer.len()
print 'Loading Review Container (%s)' %YRFILE
reviewContainer = pickle.load( open( YRFILE, "rb" ) )
print '\t-> loaded %d reviews' %reviewContainer.len()
from data.food_dataset.trieFinder import *
## load trie
trie = json.load(open('data/food_dataset/bbc_menupages-trie.json'))
trie_finder = TrieFinder(trie)
print 'loaded trie'

Loading Business Container (data/yelp_restaurants_business.p)
	-> loaded 21892 business
Loading Review Container (data/yelp_restaurants_review.p)
	-> loaded 990627 reviews
loaded trie


In [268]:
'''
Parse Review Objects searching for food items
'''
print >> sys.stderr, 'Searching food-items in reviews'
food_sentences = 0
rev_with_food = 0
rev_no_text = 0
rev_no_food = 0
lines = 0
for revObj in reviewContainer.entries:
  rev = revObj.text
  print "------"
  if len(rev.split(' ')) < 3:
    rev_no_text += 1
    print "[!] no text"
    continue
  lines += 1
  ## splint in sentences, then in words
  for sent in tokenize2sentences2words( rev ):
    l_words = remove_stopwords( sent )
    ## select only nouns from l_words
    l_nouns = filter_POS_type( l_words, 'NN' )
    ## match each word with food
    matches = ''
    topjac = -1
    ##
    nouns_food_tmp = trie_finder.find(l_words)
    ## clean:
    nouns_food = []
    for nf in nouns_food_tmp:
        if len(filter_POS_type(nf, 'NN')) > 0:
            nouns_food.append(nf)
    if len(nouns_food) > 0:
        emot = myLIWC.getCategoriesFromText(' '.join(sent))
        if emot['posemo'] != emot['negemo']:
            print ' '.join(sent), ':'
            print '\t->', nouns_food, '->', emot
  ##
  if lines >= 10:
    break
  if lines % 100 == 0:
    print >> sys.stderr, '\t-> found %s rev_with_food (%s tot_food_sentences, %d rev_no_food, %d rev_no_text)' \
    %(rev_with_food, food_sentences, rev_no_food, rev_no_text)
##
print >> sys.stderr, '\t-> found %s rev_with_food (%s tot_food_sentences, %d rev_no_food, %d rev_no_text)' \
%(rev_with_food, food_sentences, rev_no_food, rev_no_text)

------
the fish was very good but the reuben was to die for :
	-> [u'fish', u'reuben'] -> {'posemo': 1, 'negemo': 0}
both dishes were massive and could very easily be shared between two people :
	-> [u'two'] -> {'posemo': 2, 'negemo': 0}
------
cant miss stop for the best fish sandwich in pittsburgh :
	-> [u'fish sandwich'] -> {'posemo': 2, 'negemo': 0}
------
fish sandwiich salmon huge and delicious flounder shrimp a few ways norfolk style is oily for my taste and i never had it growing up in norfolk :
	-> [u'fish', u'salmon', u'shrimp'] -> {'posemo': 1, 'negemo': 0}
hawkins st special prime rib sized for two watch it the prices are low the portions are large and just about everything on the menu is delicious :
	-> [u'special', u'prime rib', u'two'] -> {'posemo': 2, 'negemo': 1}
------
i had the fish sandwich which was great :
	-> [u'fish sandwich'] -> {'posemo': 1, 'negemo': 0}
------
the best fish and chips youll ever enjoy and equally superb fried shrimp :
	-> [u'fish', u'fried shr

Searching food-items in reviews
	-> found 0 rev_with_food (0 tot_food_sentences, 0 rev_no_food, 0 rev_no_text)

 :
	-> [u'fish sandwich'] -> {'posemo': 2, 'negemo': 0}


In [260]:
sent = 'both dishes were massive large and two could very easily be shared between two people : \
cant miss stop for the best fish sandwich in pittsburgh'.split(' ')
l_words = remove_stopwords( sent )
print '---'
print l_words
print trie_finder.find(l_words)
## select only nouns from l_words
l_nouns = filter_POS_type( l_words, 'NN' )
print '---'
print l_nouns
print trie_finder.find(l_nouns)
print '---'
cleaning = []
for tf in trie_finder.find(l_words):
    if len(filter_POS_type(tf,'NN')) > 0:
        cleaning.append(tf)
print cleaning

---
['dishes', 'massive', 'large', 'two', 'could', 'easily', 'shared', 'two', 'people', ':', 'cant', 'miss', 'stop', 'best', 'fish', 'sandwich', 'pittsburgh']
['two', 'two', 'fish sandwich']
---
['miss', 'stop', 'sandwich', 'pittsburgh']
['sandwich']
---
['two', 'two', 'fish sandwich']


In [34]:
## load trie
# trie = json.load(open('data/food_dataset/bbc_menupages-trie.json'))
# trie_finder = TrieFinder(trie)
# print 'loaded trie'